# Monte Carlo Scanning
## Importing Libraries

In [1]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from tqdm import tqdm
from scipy.special import erf
import pickle
import itertools

from SALib.sample import saltelli
from SALib.analyze import sobol

# Personal libraries
import sixtrackwrap as sx

from parameters import *

## Computing

In [2]:
problem = {
    'num_vars': 3,
    'names': ['alpha', 'theta1', 'theta2'],
    'bounds': [[-1, 1],
               [0, np.pi * 2],
               [0, np.pi * 2]]
    }
param_values = saltelli.sample(problem, mc_max_samples // 8 + 8)

alpha = np.array([np.arccos(p[0])/2 for p in param_values])
theta1 = np.array([p[1] for p in param_values])
theta2 = np.array([p[2] for p in param_values])

DA_3 = {}
error_3 = {}
data_3 = {}
# Data generation
engine = sx.radial_scanner(alpha, theta1, theta2, d_r, starting_step=starting_step)
engine.scan(max_turns, min_turns, batch_size=batch_size)

radiuses = engine.extract_DA(turn_sampling)

data_3 = radiuses

# Computing DA

for sample in mc_samples:
    average = np.average(np.power(radiuses[:sample], 4), axis=0)
    error = np.std(np.power(radiuses[:sample], 4), axis=0) / np.sqrt(sample)
    DA_3[sample] = np.power(average, 1/4)
    error_3[sample] = 0.25 * np.power(average, -3/4) * error
    

Active radiuses: 10064 / 10064
Sample size per active radius: 1
Expected execution time for step: nan
Elapsed time for whole iteration: 11.97
Time per single iteration: 2.3780536196758712e-05
r: 15.0 . Turns to do: 50 . Min found: 50
Active radiuses: 10064 / 10064
Sample size per active radius: 1
Expected execution time for step: 11.97
Elapsed time for whole iteration: 11.88
Time per single iteration: 2.3606130068556115e-05
r: 20.0 . Turns to do: 50 . Min found: 37
Active radiuses: 10064 / 10064
Sample size per active radius: 1
Expected execution time for step: 11.88
Elapsed time for whole iteration: 11.87
Time per single iteration: 2.359395518022425e-05
r: 25.0 . Turns to do: 50 . Min found: 3
Active radiuses: 9564 / 10064
Sample size per active radius: 1
Expected execution time for step: 11.28
Elapsed time for whole iteration: 17.86
Time per single iteration: 3.73414828158083e-05
r: 30.0 . Turns to do: 50 . Min found: 0
Active radiuses: 5834 / 10064
Sample size per active radius: 1
E

### Saving data

In [3]:
with open(savepath + "data/DA_3.pkl", 'wb') as f:
    pickle.dump(DA_3, f, protocol=4)
    
with open(savepath + "data/error_3.pkl", 'wb') as f:
    pickle.dump(error_3, f, protocol=4)